In [49]:
import os
os.environ['CC'] = os.path.join(os.environ['CONDA_PREFIX'], 'bin', 'x86_64-conda-linux-gnu-gcc')
os.environ['CXX'] = os.path.join(os.environ['CONDA_PREFIX'], 'bin', 'x86_64-conda-linux-gnu-g++')
import torch
print(torch.version.cuda)
from PIL import Image
import requests
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor
from datasets import load_dataset
from evaluate import load
import pandas as pd
import os
from peft import LoraConfig, get_peft_model
from transformers import TrainerCallback
from datasets import Dataset, Features, Array2D
from torch.utils.data import DataLoader
import logging


if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Number of GPUs: {torch.cuda.device_count()}")

    # List details for each GPU
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    torch.cuda.empty_cache()  # Clears unused memory
    torch.cuda.reset_max_memory_allocated()
import sys
print(f"Python version: {sys.version}")

    
#export PATH=/home/koehler.ale/.conda/envs/transform/bin:$PATH

    


12.1
CUDA is available!
Number of GPUs: 1
GPU 0: NVIDIA A100-SXM4-80GB
Python version: 3.10.15 (main, Oct  3 2024, 07:27:34) [GCC 11.2.0]


/home/koehler.ale/.conda/envs/change/lib/python3.10/site-packages/torch/cuda/memory.py:365: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [50]:
import os

# Print current CONDA_PREFIX to see what it's set to
print(f"Current CONDA_PREFIX: {os.environ.get('CONDA_PREFIX', 'Not set')}")

# We know the compiler exists here from our earlier tests
expected_prefix = "/home/koehler.ale/.conda/envs/change"

# Override CONDA_PREFIX if it's wrong
if os.environ.get('CONDA_PREFIX') != expected_prefix:
    os.environ['CONDA_PREFIX'] = expected_prefix

conda_gcc = os.path.join(os.environ['CONDA_PREFIX'], 'bin', 'x86_64-conda-linux-gnu-gcc')
conda_gxx = os.path.join(os.environ['CONDA_PREFIX'], 'bin', 'x86_64-conda-linux-gnu-g++')

# Verify paths exist
if not os.path.exists(conda_gcc) or not os.path.exists(conda_gxx):
    raise FileNotFoundError(f"GCC/G++ not found at expected paths: {conda_gcc}, {conda_gxx}")

# Set environment variables
os.environ['CC'] = conda_gcc
os.environ['CXX'] = conda_gxx
os.environ['GCC_PATH'] = os.path.dirname(conda_gcc)

# Print paths to verify
print(f"Using GCC at: {os.environ['CC']}")
print(f"Using G++ at: {os.environ['CXX']}")

Current CONDA_PREFIX: /home/koehler.ale/.conda/envs/change
Using GCC at: /home/koehler.ale/.conda/envs/change/bin/x86_64-conda-linux-gnu-gcc
Using G++ at: /home/koehler.ale/.conda/envs/change/bin/x86_64-conda-linux-gnu-g++


In [51]:
import torch
import os

# Check CUDA availability and version
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Torch version: {torch.__version__}")

# Get device properties
if torch.cuda.is_available():
    print(f"Current device: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name()}")
    
# Check CUDA paths
print("\nCUDA paths:")
print(f"PyTorch installation: {torch.__file__}")
print(f"CUDA_HOME environment variable: {os.environ.get('CUDA_HOME')}")
print(f"Current LD_LIBRARY_PATH: {os.environ.get('LD_LIBRARY_PATH')}")

CUDA available: True
CUDA version: 12.1
Torch version: 2.5.1+cu121
Current device: 0
Device name: NVIDIA A100-SXM4-80GB

CUDA paths:
PyTorch installation: /home/koehler.ale/.conda/envs/change/lib/python3.10/site-packages/torch/__init__.py
CUDA_HOME environment variable: /shared/centos7/cuda/12.1
Current LD_LIBRARY_PATH: /shared/centos7/cuda/12.1/lib64:/shared/centos7/anaconda3/3.7/lib:/shared/centos7/nodejs/14.15.4/lib


In [52]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit", # Llama 3.2 vision support
    "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
    "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit", # Can fit in a 80GB card!
    "unsloth/Llama-3.2-90B-Vision-bnb-4bit",

    "unsloth/Pixtral-12B-2409-bnb-4bit",              # Pixtral fits in 16GB!
    "unsloth/Pixtral-12B-Base-2409-bnb-4bit",         # Pixtral base model

    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",          # Qwen2 VL support
    "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
    "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",

    "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",      # Any Llava variant works!
    "unsloth/llava-1.5-7b-hf-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

==((====))==  Unsloth 2024.11.10: Fast Mllama vision patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-80GB. Max memory: 79.15 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.59s/it]


In [53]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

In [58]:
from datasets import load_from_disk

def load_scratch_dataset():
    """
    Load the dataset from scratch directory
    
    Returns:
        Dataset: Loaded dataset with PIL images
    """
    load_path = "/scratch/koehler.ale/xray_dataset"
    
    if not os.path.exists(load_path):
        raise FileNotFoundError(f"No dataset found at {load_path}")
    
    print(f"Loading dataset from {load_path}")
    dataset = load_from_disk(load_path)
    print(f"Loaded dataset with {len(dataset)} examples")
    print(f"First image type: {type(dataset[0]['image'])}")
    print(dataset[0]['Content'])
    print(dataset.column_names)
    return dataset

dataset = load_scratch_dataset()
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

# Extract the training and validation sets
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(train_dataset.column_names)

Loading dataset from /scratch/koehler.ale/xray_dataset
Loaded dataset with 4374 examples
First image type: <class 'PIL.PngImagePlugin.PngImageFile'>
Moderate right pleural effusion has changed in distribution, but not in
 overall volume since ___.  Some of it may now be loculated, particular
 the non dependent portions in the right upper chest.  No pneumothorax.  Right
 upper lobe is clear.  Right lower lobe is restricted by the pleural
 abnormality, are better evaluated by either con current chest CT reported
 separately.  Heart is moderately enlarged.  Left lung is clear.

['Content', 'image']
Training set size: 3499
Validation set size: 875
['Content', 'image']


In [55]:
instruction = "You are an expert radiographer. Describe accurately what you see in this image. Do not be vague, describe each detail"

def convert_to_conversation(sample):
    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : sample["image"]} ]
        },
        { "role" : "assistant",
          "content" : [
            {"type" : "text",  "text"  : sample["Content"]} ]
        },
    ]
    return { "messages" : conversation }
pass

In [59]:
train_dataset = [convert_to_conversation(sample) for sample in train_dataset]
val_dataset = [convert_to_conversation(sample) for sample in val_dataset]

In [60]:
train_dataset[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'You are an expert radiographer. Describe accurately what you see in this image. Do not be vague, describe each detail'},
    {'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=224x224>}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': '1.  Prominent cardiac silhouette.\n 2.  No displaced rib fracture.\n 3.  No significant pleural effusion.\n'}]}]}

In [61]:
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

# Your training code
FastVisionModel.for_training(model) # Enable for training!
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer),
    train_dataset = train_dataset,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,
        learning_rate = 2e-4,
        fp16 = not is_bf16_supported(),
        bf16 = is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        dataset_num_proc = 4,
        max_seq_length = 2048,
    ),
)


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


In [62]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-80GB. Max memory = 79.15 GB.
26.281 GB of memory reserved.


In [ ]:
# trainer_stats = trainer.train()

In [ ]:
# #@title Show final memory and time stats
# used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
# used_percentage = round(used_memory         /max_memory*100, 3)
# lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
# print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
# print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
# print(f"Peak reserved memory = {used_memory} GB.")
# print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
# print(f"Peak reserved memory % of max memory = {used_percentage} %.")
# print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
FastVisionModel.for_inference(model) # Enable for inference!

image = val_dataset[30]["image"]
instruction = "You are an expert radiographer. Describe accurately what you see in this image."

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

In [83]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=UnslothVisionDataCollator(model, tokenizer),
    train_dataset=train_dataset,
    eval_dataset=val_dataset, 
    args=SFTConfig(
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=30,
        learning_rate=2e-4,
        fp16=not is_bf16_supported(),
        bf16=is_bf16_supported(),
        logging_steps=1,
        evaluation_strategy="steps",  # Run evaluation during training
        eval_steps=5,  # Run evaluation every 5 steps
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
        remove_unused_columns=False,
        dataset_text_field="",
        dataset_kwargs={"skip_prepare_dataset": True},
        dataset_num_proc=4,
        max_seq_length=2048,
    ),
)

# Train the model with validation
trainer.train()


/home/koehler.ale/.conda/envs/change/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,499 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 30
 "-____-"     Number of trainable parameters = 67,174,400
🦥 Unsloth needs about 1-3 minutes to load everything - please wait!


Step,Training Loss,Validation Loss
5,0.120200,0.818936
10,0.161400,1.022932
15,0.231000,0.925203
20,0.374400,0.796682
25,0.510300,0.730458
30,0.414700,0.706344


TrainOutput(global_step=30, training_loss=0.3695407651364803, metrics={'train_runtime': 2387.4762, 'train_samples_per_second': 0.101, 'train_steps_per_second': 0.013, 'total_flos': 1030320889011828.0, 'train_loss': 0.3695407651364803, 'epoch': 0.06857142857142857})

In [82]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from torch.utils.data import DataLoader

# Evaluation function using cosine similarity
def evaluate_with_cosine_similarity(model, tokenizer, eval_dataloader):
    predictions = []
    actual_content = []

    # Generate predictions
    model.eval()  # Set the model to evaluation mode
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model.generate(**batch)  # Generate predictions
        preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        labels = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)
        
        predictions.extend(preds)
        actual_content.extend(labels)

    # Calculate embeddings using Sentence-BERT
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2')  # You can change this model as needed
    pred_embeddings = sbert_model.encode(predictions, convert_to_tensor=True)
    ref_embeddings = sbert_model.encode(actual_content, convert_to_tensor=True)

    # Compute cosine similarity
    similarities = cosine_similarity(pred_embeddings.cpu().numpy(), ref_embeddings.cpu().numpy())
    diagonal_similarities = np.diagonal(similarities)  # One-to-one comparison

    # Calculate average similarity
    avg_similarity = np.mean(diagonal_similarities)

    print("\nCosine Similarity Results:")
    print(f"Average Cosine Similarity: {avg_similarity:.4f}")
    return predictions, actual_content, avg_similarity

# Prepare eval_dataloader
eval_dataloader = DataLoader(val_dataset, batch_size=2)  # Adjust batch size as needed

# Evaluate the fine-tuned model
predictions, actual_content, avg_similarity = evaluate_with_cosine_similarity(
    model=trainer.model,  # Use the fine-tuned model from SFTTrainer
    tokenizer=tokenizer,
    eval_dataloader=eval_dataloader
)

print(f"Average Cosine Similarity: {avg_similarity:.4f}")


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.PngImagePlugin.PngImageFile'>